### Example Exploratory Notebook

Use this notebook to explore the data generated by the pipeline in your preferred programming language.

**Note**: This notebook is not executed as part of the pipeline.

In [0]:
# Create a streaming DataFrame by reading from the profiles table
df = (
  spark.table("profiles_bronze")
    .select("topic", "partition", "offset", "value_str", "timestampType", "timestamp", "ingestTime")
    .withColumn(
      "latency_ms"
      ,unix_millis(col("ingestTime")) - unix_millis(col("timestamp"))
    )
    .orderBy(col("timestamp"), ascending=False)
    .limit(1000)
  )

# Display the streaming DataFrame
display(df)

In [0]:
# Calculate the average latency_ms from streaming_df
average_latency_ms_df = df.groupBy().avg("latency_ms")

# Display the result
display(average_latency_ms_df)

In [0]:
df = (
  spark.read.option("readChangeFeed", "true")
  .option("startingVersion", 11)
  .table(f"profiles_bronze")
  .filter("_change_type = 'delete'")
)
display(df)

In [0]:
%sql
DESCRIBE EXTENDED profiles AS JSON;

In [0]:
%sql 
FROM profiles_scd2 |>
AGGREGATE COUNT(*) as rcrd_cnt 
GROUP BY __START_AT._commit_version 

In [0]:
%fs ls s3://adc-lv-dev-devops.libreview-data-migration/patient-1_Measurement/patient-1_Measurement/

In [0]:
%fs  head -100 s3://adc-lv-dev-devops.libreview-data-migration/patient-1_Measurement/patient-1_Measurement/patient-1_Measurement_1.tsv

In [0]:
%fs ls s3://adc-lv-dev-devops.libreview-data-migration/

In [0]:
# bronze_patientpractice
# D
# {"practiceId":"b10e40f5-5521-42c1-9b6f-4f18c74cd940","shard":2007,"patientId":"070470ae-be33-4ed8-a42a-01b4b89051ff","referrerId":"9621d57e-e27d-e711-818b-06f1e198757d","name":"3k Patients","address1":"Alafaya","address2":"","city":"Orlando","state":"FL","zipCode":"32826","country":"US","phoneNumber":"4075555555","created":1502379813,"createdBy":"migrate-lv","updated":1571712004,"updatedBy":"migrate-lv"}

# {"address1":"852 Wilderman Spring","address2":"2226 Emard Fields","anonymous":false,"businessId":"idr8xjgufr","city":"West Mozellemouth","country":"US","created":1586286353,"createdBy":"cda040f5-7902-11ea-858e-0242ac110002","name":"practice0w66gx5w","patientId":"cda040f5-7902-11ea-858e-0242ac110002","phoneNumber":"01651369554","practiceId":"cccd9346-7902-11ea-858e-0242ac110002","referrerId":"c9a6ff49-7902-11ea-b9ce-0242ac11000a","shard":325,"state":"South Carolina","updated":1586286353,"updatedBy":"cda040f5-7902-11ea-858e-0242ac110002","zipCode":"50224"}


# P
# {"b10e40f5-5521-42c1-9b6f-4f18c74cd940":"w","070470ae-be33-4ed8-a42a-01b4b89051ff":"w"}

In [0]:
# bronze_measurement
#{"d1":22,"d8":0,"d31":70,"d32":250,"d33":true,"d34":true,"d35":true}

# {"shard":1465,"id":"7230f654-de04-11ed-8f95-0242ac110006","p":"71514a73-de04-11ed-892b-0242ac110002","did":"722ede0a-de04-11ed-8f95-0242ac110006","uid":"7223f647-de04-11ed-a5fc-0242ac110006","ts":"2023-04-18T11:16:59.321-05:00","t":6,"fts":"2023-04-18T16:16:59.321Z","rn":775,"created":1681834619,"d1":22,"d2":"","d8":0,"d31":70,"d32":250,"f":{},"l":{},"h":{},"s":{}}

In [0]:
# pipelines.streamingFlowReadOptionsEnabled
# spark.databricks.streaming.realTimeMode.enabled

In [0]:
# Query [id = 32a5f0cc-af4a-479b-9c11-4dde4f41fa3a, runId = 2fa934e8-e59b-4692-a2e5-240b0932a63d] terminated with exception: [MALFORMED_RECORD_IN_PARSING.WITHOUT_SUGGESTION] Malformed records are detected in record parsing: {.
# Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.  SQLSTATE: 22023

In [0]:
# Cannot resolve "semi_structured_extract_json_multi(D_variant, $.connectionId)" due to data type mismatch: The first parameter requires the "STRING" type, however "D_variant" has the type "STRUCT<connectionId: STRING, patientId: STRING, status: STRING, source: STRING, created: BIGINT, createdBy: STRING, updated: BIGINT, updatedBy: STRING>".